In [1]:
import numpy as np
import gymnasium as gym
import random
import time
from IPython.display import clear_output

In [2]:
env=gym.make('FrozenLake-v1', render_mode="rgb_array")

In [3]:
action_size = env.action_space.n
state_size = env.observation_space.n

In [4]:
qtable = np.zeros((state_size, action_size))
print(qtable)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [5]:
print(action_size)
print(state_size)

4
16


In [6]:
total_episodes = 10000        # Total episodes
learning_rate = 0.1           # Learning rate
max_steps = 100                # Max steps per episode
gamma = 0.99                  # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.001            # Exponential decay rate for exploration prob

# **Train Q-learning Agent**

In [7]:
# List of rewards
rewards = []

# 2 For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()[0]
    step = 0
    done = False
    total_rewards = 0
    
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()

        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done,truncated, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])
        
        total_rewards += reward
        
        # Our new state is state
        state = new_state
        
        # If done (if we're dead) : finish episode
        if done == True: 
            break
        
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)

print ("Score over time: " +  str(sum(rewards)/total_episodes))
print("\n\n********Q-table********\n")
print(qtable)

Score over time: 0.5176


********Q-table********

[[0.58463587 0.51716657 0.53942822 0.51167857]
 [0.34387883 0.2772464  0.35341456 0.53961632]
 [0.41211697 0.41652149 0.41465908 0.49963768]
 [0.24896619 0.31298287 0.28788306 0.4573954 ]
 [0.60303535 0.40136654 0.44562854 0.41566184]
 [0.         0.         0.         0.        ]
 [0.16420679 0.1342681  0.39265738 0.11683524]
 [0.         0.         0.         0.        ]
 [0.40946559 0.51383015 0.3460786  0.64215524]
 [0.3553397  0.71697678 0.43511846 0.43228122]
 [0.67830353 0.38518766 0.38342885 0.31908218]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.51207722 0.42110326 0.78207868 0.56496755]
 [0.72613775 0.95031387 0.72056574 0.78615191]
 [0.         0.         0.         0.        ]]


In [8]:
rewards_per_thousand_episodes = np.split(np.array(rewards),total_episodes/1000)
count = 1000

print("********Average reward per thousand episodes********\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

********Average reward per thousand episodes********

1000 :  0.027000000000000017
2000 :  0.20800000000000016
3000 :  0.3870000000000003
4000 :  0.5530000000000004
5000 :  0.6520000000000005
6000 :  0.6280000000000004
7000 :  0.6690000000000005
8000 :  0.6590000000000005
9000 :  0.6980000000000005
10000 :  0.6950000000000005


In [22]:

for episode in range(3):
    state = env.reset()[0]
    done = False
    print("****************************************************")
    print("EPISODE ", episode+1)
    time.sleep(1)
    for step in range(max_steps):
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, truncated, info = env.step(action)
        
        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1:
                print("****You reached the goal!****")
                time.sleep(3)
            else:
                print("****You fell through a hole!****")
                time.sleep(3)
            clear_output(wait=True)
            break
        state=new_state
env.close()

****You reached the goal!****
